# Review of A/B Testing Tools

- Sample size selection for proportions
  - Single treatment
  - Multitreatment
- Hypothesis test for proportions
- Power analysis for proportions

## Review of Tests

**t-test**

- https://en.wikipedia.org/wiki/Student%27s_t-test#Assumptions
- is used for continuous values.
- one-sided assumes this will be from the t dist: $\frac{ \hat{\mu} - \mu_0 }{ \hat{\sigma} / \sqrt{n} }$
- 2 dist
  - means are normal (valid with enough data due to CLT -- means always converge)
  - same std
  - same size
  - usually still works unless IID assumption is broken

**z-test**

- TODO

**chi-square**

- TODO

### Key Terms

- Power is the likelihood h0 is rejected given that h1 is real
- P-value is the probability of seeing a more extreme result given null hypot is true

### Resources

- https://machinelearningmastery.com/effect-size-measures-in-python/
- http://jpktd.blogspot.com/2013/03/statistical-power-in-statsmodels.html
- https://www.evanmiller.org/ab-testing/sample-size.html

In [2]:
import statsmodels.stats.power as pwr

In [4]:
?pwr.ttest_power

Signature: pwr.ttest_power(effect_size, nobs, alpha, df=None, alternative='two-sided')
Docstring:
Calculate power of a ttest
    
File:      ~/Documents/learn-python/.venv/lib/python3.7/site-packages/statsmodels/stats/power.py
Type:      function


In [3]:
pwr.ttest_power(0.2, nobs=60, alpha=0.1, alternative='two-sided')

0.45558175996348543

In [7]:
tt_pwr = pwr.TTestIndPower()
?tt_pwr.solve_power

Signature:
tt_pwr.solve_power(
    effect_size=None,
    nobs1=None,
    alpha=None,
    power=None,
    ratio=1.0,
    alternative='two-sided',
)
Docstring:
solve for any one parameter of the power of a two sample t-test

for t-test the keywords are:
    effect_size, nobs1, alpha, power, ratio

exactly one needs to be ``None``, all others need numeric values

Parameters
----------
effect_size : float
    standardized effect size, difference between the two means divided
    by the standard deviation. `effect_size` has to be positive.
nobs1 : int or float
    number of observations of sample 1. The number of observations of
    sample two is ratio times the size of sample 1,
    i.e. ``nobs2 = nobs1 * ratio``
alpha : float in interval (0,1)
    significance level, e.g. 0.05, is the probability of a type I
    error, that is wrong rejections if the Null Hypothesis is true.
power : float in interval (0,1)
    power of the test, e.g. 0.8, is one minus the probability of a
    type II erro

In [5]:
pwr.TTestIndPower().solve_power(0.3, power=0.75, ratio=1, alpha=0.05, alternative='larger')

120.22320283709918

Mini-example

- 500 loans a day
- Want to try new terms
- Want to see difference in default rates at least 1 percentage point
- Current default rate is 10%

In [37]:
import numpy as np

p = 0.5 # worst case for binomial, when each 
p = 0.15 # assuming null hypot, no change
std_dev = np.sqrt( p * (1-p) )
diff = 0.01
effect_size = diff / std_dev

pwr.TTestIndPower().solve_power(
    effect_size=effect_size,
    alpha=0.05,
    power=0.8,
    ratio=1,
    alternative='two-sided',
)

20015.555002803725

In [40]:
pwr.zt_ind_solve_power(
    effect_size=effect_size,
    alpha=0.05,
    power=0.8,
    ratio=1,
    alternative='two-sided',
)

20014.59429880191

In [46]:
chi_pwr = pwr.GofChisquarePower()
?chi_pwr.solve_power

Signature:
chi_pwr.solve_power(
    effect_size=None,
    nobs=None,
    alpha=None,
    power=None,
    n_bins=2,
)
Docstring:
solve for any one parameter of the power of a one sample chisquare-test

for the one sample chisquare-test the keywords are:
    effect_size, nobs, alpha, power

Exactly one needs to be ``None``, all others need numeric values.

n_bins needs to be defined, a default=2 is used.


Parameters
----------
effect_size : float
    standardized effect size, according to Cohen's definition.
    see :func:`statsmodels.stats.gof.chisquare_effectsize`
nobs : int or float
    sample size, number of observations.
alpha : float in interval (0,1)
    significance level, e.g. 0.05, is the probability of a type I
    error, that is wrong rejections if the Null Hypothesis is true.
power : float in interval (0,1)
    power of the test, e.g. 0.8, is one minus the probability of a
    type II error. Power is the probability that the test correctly
    rejects the Null Hypothesis if

In [48]:
import statsmodels
?statsmodels.stats.gof.chisquare_effectsize

Signature:
statsmodels.stats.gof.chisquare_effectsize(
    probs0,
    probs1,
    correction=None,
    cohen=True,
    axis=0,
)
Docstring:
effect size for a chisquare goodness-of-fit test

Parameters
----------
probs0 : array_like
    probabilities or cell frequencies under the Null hypothesis
probs1 : array_like
    probabilities or cell frequencies under the Alternative hypothesis
    probs0 and probs1 need to have the same length in the ``axis`` dimension.
    and broadcast in the other dimensions
    Both probs0 and probs1 are normalized to add to one (in the ``axis``
    dimension).
correction : None or tuple
    If None, then the effect size is the chisquare statistic divide by
    the number of observations.
    If the correction is a tuple (nobs, df), then the effectsize is
    corrected to have less bias and a smaller variance. However, the
    correction can make the effectsize negative. In that case, the
    effectsize is set to zero.
    Pederson and Johnson (1990) as ref

In [51]:
effect_size

0.028005601680560196

In [52]:
statsmodels.stats.gof.chisquare_effectsize(
    probs0=[0.15, 0.85],
    probs1=[0.16, 0.84],
)

0.028005601680560224

In [53]:
statsmodels.stats.gof.chisquare_effectsize(
    probs0=[0.15, 0.85],
    probs1=[0.14, 0.87],
)

0.03188756626994474

In [55]:
pwr.GofChisquarePower().solve_power(
    effect_size=effect_size,
    alpha=0.05,
    power=0.8,
)

10007.29714939089

## Hypothesis Test

In [68]:
import statsmodels.stats.api as sm

results1 = [
    [5000, 3081], # A
    [4000, 2700], # B
]

results1[0][0] / sum(results1[0]), results1[1][0] / sum(results1[1])

(0.6187353050365054, 0.5970149253731343)

In [58]:
?sm.proportions_ztest

Signature:
sm.proportions_ztest(
    count,
    nobs,
    value=None,
    alternative='two-sided',
    prop_var=False,
)
Docstring:
Test for proportions based on normal (z) test

Parameters
----------
count : {int, array_like}
    the number of successes in nobs trials. If this is array_like, then
    the assumption is that this represents the number of successes for
    each independent sample
nobs : {int, array_like}
    the number of trials or observations, with the same length as
    count.
value : float, array_like or None, optional
    This is the value of the null hypothesis equal to the proportion in the
    case of a one sample test. In the case of a two-sample test, the
    null hypothesis is that prop[0] - prop[1] = value, where prop is the
    proportion in the two samples. If not provided value = 0 and the null
    is prop[0] = prop[1]
alternative : str in ['two-sided', 'smaller', 'larger']
    The alternative hypothesis can be either two-sided or one of the one-
    sided

In [62]:
sm.proportions_ztest(
    count=[results1[0][0], results1[1][0]], # Successes
    nobs=[sum(results1[0]), sum(results1[1])],
    value=0, # null is no diff
    alternative='two-sided',
    prop_var=False,
)

(2.693807365822314, 0.007064097883455329)

In [64]:
sm.proportions_ztest(
    count=[results1[0][0], results1[1][0]], # Successes
    nobs=[sum(results1[0]), sum(results1[1])],
    value=0, # null is no diff
    alternative='two-sided',
    prop_var=results1[0][0] / sum(results1[0]), # proportion of control, sample A
)

(2.7065728058306524, 0.006798167406192918)

In [65]:
sm.proportions_ztest(
    count=[results1[0][0], results1[1][0]], # Successes
    nobs=[sum(results1[0]), sum(results1[1])],
    value=0, # null is no diff
    alternative='larger',
    prop_var=results1[0][0] / sum(results1[0]), # proportion of control, sample A
)

(2.7065728058306524, 0.003399083703096459)

In [69]:
sm.proportions_ztest(
    count=[results1[0][0], results1[1][0]], # Successes
    nobs=[sum(results1[0]), sum(results1[1])],
    value=0, # null is no diff
    alternative='smaller',
    prop_var=results1[0][0] / sum(results1[0]), # proportion of control, sample A
)

(2.7065728058306524, 0.9966009162969035)

In [74]:
sm.proportions_chisquare(
    count=[results1[0][0], results1[1][0]], # Successes
    nobs=[sum(results1[0]), sum(results1[1])],
)

(7.256598124158506,
 0.007064097883455526,
 (array([[5000, 3081],
         [4000, 2700]]),
  array([[4920.43840065, 3160.56159935],
         [4079.56159935, 2620.43840065]])))

In [81]:
sm.ttest_ind(
    x1= [0] * results1[0][0] + [1] * results1[0][1],
    x2= [0] * results1[1][0] + [1] * results1[1][1],
    alternative='two-sided',
    usevar='pooled',
    weights=(None, None),
    value=0,
)

(-2.694286560668368, 0.007061918115177303, 14779.0)

All look very similar....

## Power Calculations

In [95]:
p0 = results1[0][0] / sum(results1[0])
p1 = results1[1][0] / sum(results1[1])
diff = p0 - p1
nobs1 = sum(results1[0])
ratio = nobs1 / sum(results1[1])
std = np.sqrt( p0 * (1 - p0) )
effect_size = diff / std

sm.tt_ind_solve_power(
    effect_size=effect_size,
    nobs1=nobs1,
    ratio=ratio,
    alpha=0.5,
    alternative='two-sided',
), effect_size, diff

(0.989350186020283, 0.044719986905560935, 0.021720379663371148)

In [97]:
sm.zt_ind_solve_power(
    effect_size=effect_size,
    nobs1=nobs1,
    ratio=ratio,
    alpha=0.5,
    alternative='two-sided',
)

0.9893507241486387